In [1]:

#model selection and evaluation
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor
import matplotlib.pyplot as plt

from glob import glob
 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
#import cv2
import gc
import os
 
import tensorflow as tf

 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import rdkit
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend
from tensorflow.keras.layers import   Dense,Flatten, Reshape, LeakyReLU, Dropout,InputLayer,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from rdkit import Chem
sns.set_style(style="darkgrid")
%matplotlib inline

In [2]:
!pip install scikeras


In [3]:

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

# DRUG DISCOVERY PROCESS

In [4]:
data = pd.read_csv("data.csv", error_bad_lines=False,sep=";")

MemoryError: Unable to allocate 128. KiB for an array with shape (16384,) and data type int64

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.info()

# Missing Values

In [ ]:
data.isna().sum()

In [ ]:
data=data.replace('None',np.nan)

In [ ]:
data=data.replace('',np.nan)

In [ ]:
data.isna().sum()

In [ ]:
def missing_values_percentage_by_column(df):
    cols = df.columns.tolist()
    missing_percentage = {}
    for col in cols:
        total_cells = df[col].size
        missing_cells = df[col].isnull().sum()
        percentage = (missing_cells / total_cells) * 100
        missing_percentage[col] = percentage
    return missing_percentage

In [ ]:
missing_values_percentage_by_column(data)

In [ ]:
def plot_missing_values(df):
    missing_df = pd.DataFrame.from_dict(missing_values_percentage_by_column(data), orient='index', columns=['percentage'])
    missing_df = missing_df.sort_values(by='percentage', ascending=False)

    plt.figure(figsize=(10, 6))
    plt.bar(missing_df.index, missing_df['percentage'])
    plt.xticks(rotation=90)
    plt.ylabel('% of Missing Values')
    plt.title('Missing Value Distribution by Column')
    plt.show()

In [ ]:
plot_missing_values(data)

# Dealing with useless Features and Missing Values

In [ ]:
data=data.drop(['Synonyms','Name','Max Phase'],axis=1)

In [ ]:
data=data.drop(['ChEMBL ID','Type'],axis=1)

In [ ]:
data=data.dropna(axis=0,how='any')

In [ ]:
data.isna().sum()

# Duplicate Values-Unique Values

In [ ]:
data.duplicated().sum()

In [ ]:
data.nunique()

In [ ]:
data.shape

In [ ]:

data

# Data Transformation

In [ ]:
data['Targets'] = pd.to_numeric(data['Targets'], errors='coerce')
data['Bioactivities'] = pd.to_numeric(data['Bioactivities'], errors='coerce')
data['AlogP'] = pd.to_numeric(data['AlogP'], errors='coerce')
data['Polar Surface Area'] = pd.to_numeric(data['Polar Surface Area'], errors='coerce')
data['Inorganic Flag'] = pd.to_numeric(data['Inorganic Flag'], errors='coerce')
data['Heavy Atoms'] = pd.to_numeric(data['Heavy Atoms'], errors='coerce')
data['HBA (Lipinski)'] = pd.to_numeric(data['HBA (Lipinski)'], errors='coerce')
data['HBD (Lipinski)'] = pd.to_numeric(data['HBD (Lipinski)'], errors='coerce')
data['#RO5 Violations (Lipinski)'] = pd.to_numeric(data['#RO5 Violations (Lipinski)'], errors='coerce')
data['Molecular Weight (Monoisotopic)'] = pd.to_numeric(data['Molecular Weight (Monoisotopic)'], errors='coerce')
data['HBA'] = pd.to_numeric(data['HBA'], errors='coerce')
data['HBD'] = pd.to_numeric(data['HBD'], errors='coerce')
data['#RO5 Violations'] = pd.to_numeric(data['#RO5 Violations'], errors='coerce')
data['#Rotatable Bonds'] = pd.to_numeric(data['#Rotatable Bonds'], errors='coerce')
data['QED Weighted'] = pd.to_numeric(data['QED Weighted'], errors='coerce')
data['CX Acidic pKa'] = pd.to_numeric(data['CX Acidic pKa'], errors='coerce')
data['CX Basic pKa'] = pd.to_numeric(data['CX Basic pKa'], errors='coerce')
data['CX LogP'] = pd.to_numeric(data['CX LogP'], errors='coerce')
data['CX LogD'] = pd.to_numeric(data['CX LogD'], errors='coerce')
data['Aromatic Rings'] = pd.to_numeric(data['Aromatic Rings'], errors='coerce')

In [ ]:
data.info()

# Numerical-Categorical Features

In [ ]:
cat_cols = [col for col in data.columns if data[col].dtype == 'object']
num_cols = [col for col in data.columns if data[col].dtype != 'object']

In [ ]:
plt.figure(figsize = (20, 20))
plotnumber = 1

for column in num_cols:
    if plotnumber <= 22:
        ax = plt.subplot(5, 5, plotnumber)
        sns.distplot(data[column])
        plt.xlabel(column)
        
    plotnumber += 1

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (20, 15))
plotnumber = 1

for column in ['Passes Ro3', 'Structure Type','Molecular Species']:
    if plotnumber <= 11:
        ax = plt.subplot(3, 4, plotnumber)
        sns.countplot(data[column], palette = 'rocket')
        plt.xlabel(column)
        
    plotnumber += 1

plt.tight_layout()
plt.show()

# Outilers

In [ ]:
data.shape

In [ ]:
plt.figure(figsize = (20, 20))
plotnumber = 1
for column in num_cols:
    if plotnumber <= 32:
       
        ax = plt.subplot(7, 5, plotnumber)
        sns.boxplot(data[column])
        plt.xlabel(column)
        
    plotnumber += 1

plt.tight_layout()
plt.show()

In [ ]:
#Removing Outliers using IQR
def remove_outliers_iqr(data,num_cols, factor=1.5):
    for col in num_cols:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower_threshold = q1 - factor * iqr
        upper_threshold = q3 + factor * iqr
        filtered_data = data[(data[col] >= lower_threshold) & (data[col] <= upper_threshold)]
    
    return filtered_data

In [ ]:
data1=remove_outliers_iqr(data,num_cols, factor=1.5)

In [ ]:
data1.shape

# Corrolation based feature selection

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(20,8))
mask = np.triu(np.ones_like(data.corr()))
sns.heatmap(data.corr(), annot=True, linewidths=0.2, mask=mask,cmap="Purples")

In [ ]:
# By using this function we can select correlated features
# it will remove the first feature that is correlated with anything other feature
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(data, 0.9)
corr_features

 #both CX LogP and CX LogD are computational methods for predicting the lipophilicity of a compound, 
#but CX LogD is an extension of CX LogP that takes into account the pH-dependent ionization of the molecule.
#CX LogD can be considered more precise than CX LogP because it takes into account the ionization state of the compound at a specific pH,
#which can have a significant effect on its lipophilicity.

In [ ]:
data1

In [ ]:
data1=data.drop(['#RO5 Violations','CX LogP','HBA','HBD','Molecular Weight (Monoisotopic)',
                  'Heavy Atoms'],axis=1)

In [ ]:
data1

# Feature Transformation

In [ ]:
le = LabelEncoder()

In [ ]:
data1['Structure Type'] = le.fit_transform(data1['Structure Type'])
data1['Molecular Formula'] = le.fit_transform(data1['Molecular Formula'])
data1['Molecular Species'] = le.fit_transform(data1['Molecular Species'])

In [ ]:
df2=data1.copy().head(5000)
df2

# ENCODER+DECODER+ MODEL  GAN 

In [ ]:
data1["SmileSL"]=data1["Smiles"]
data1.SmileSL=data1.SmileSL.astype("str")

In [ ]:
deuxletter=set()
for c,i in enumerate(data1["SmileSL"]):
    le=len(i)-1
    for l,j in enumerate(i):
        if((j.isalpha())&(j.isupper())&(l<le)):
                if((i[l+1].isalpha())&(i[l+1].islower())):
                    deuxletter.add(j+i[l+1])
        
        
    
                
    
print(deuxletter)

In [ ]:
remp={'Ag':'G', 'Ba':'A', 'Br':'R', 'Ca':'Q', 'Cl':'L', 'Cn':'D', 'Li':'T', 'Mg':'M', 'Na':'E', 'Sc':'X','Sn':'J','Zn':'Z',"@@":'V'}
for key,rem in remp.items():
    data1["SmileSL"]=data1.SmileSL.str.replace(key,rem)

In [ ]:
unique=set()
maxMol=0
for i in data1.SmileSL:
    
    l=len(i)
    if(l>maxMol):
        maxMol=l
    for j in i:
        unique.add(j)

In [ ]:
AtomExist={c:i for i,c in enumerate(unique)}


In [ ]:
len(data1.SmileSL)

In [ ]:
m=[]
for o,i in enumerate(data1.SmileSL):
    c=np.zeros((maxMol,len(AtomExist)))
    for j,l in enumerate(i):
            c[j,AtomExist[l]]=1
    c=c.flatten()
    m.append(c)

In [ ]:
p=m[0:26240]

In [ ]:
m=[]
for o,i in enumerate(data1.SmileSL):
    c=np.zeros((maxMol,len(AtomExist)))
    for j,l in enumerate(i):
            c[j,AtomExist[l]]=1
    c=c.flatten()
    m.append(c)

In [ ]:
unique=set()
maxMol=0
for i in data1.SmileSL:
    
    l=len(i)
    if(l>maxMol):
        maxMol=l
    for j in i:
        unique.add(j)

In [ ]:
def build_generator(): 

    generator = Sequential()
    
    generator.add(Dense(256,input_dim=128))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))
    generator.add(Dense(256))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))
    generator.add(Dense(10755, activation='sigmoid'))
    generator.add(Reshape((10755,)))
    generator.build((None, 128))
    # Conv layer to get to one channe
    
    return generator

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
def build_discriminator(): 
    
    
    discriminator = Sequential()
    discriminator.add(Dense(128, input_shape=(10755,)))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dense(64))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(0.25))
    discriminator.add(Dense(64))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.build((None, 10755))
    return discriminator 

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
rom tensorflow.keras.models import Model

In [ ]:
optimizer = Adam(0.0001, 0.5)
optimizer1 = Adam(0.00008, 0.5)
g_los = BinaryCrossentropy()
d_los = BinaryCrossentropy()

In [ ]:
class BamGAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.generator = generator 
        self.discriminator = discriminator 
         
        
    def compile(self, g_los, d_los, *args, **kwargs): 
        super().compile(*args, **kwargs)
        
        self.d_opt = Adam(0.0002, 0.5)
        self.g_opt = Adam(0.0002, 0.5)
        self.g_los = g_los
        self.d_los = d_los

    def train_step(self, batch):
        print(batch)
        real = batch
        fake = self.generator(tf.random.normal((128, 128)), training=False)
        with tf.GradientTape() as d_tape: 
            ypr_real = self.discriminator(real, training=True) 
            ypr_fake = self.discriminator(fake, training=True)
            ypr_all = tf.concat([ypr_real, ypr_fake], axis=0)
            y_all = tf.concat([tf.zeros_like(ypr_real), tf.ones_like(ypr_fake)], axis=0) 
            total_d_los = self.d_los(y_all, ypr_all)
        dgrad = d_tape.gradient(total_d_los, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables)) 
        with tf.GradientTape() as g_tape: 
            gen_smil = self.generator(tf.random.normal((128,128)), training=True)
            predicted_labels = self.discriminator(gen_smil, training=False)
            total_g_los = self.g_los(tf.zeros_like(predicted_labels), predicted_labels) 
        ggrad = g_tape.gradient(total_g_los, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        return {"d_loss":total_d_los, "g_loss":total_g_los}

In [ ]:
bamgan = BamGAN(generator, discriminator)
bamgan.compile( g_los, d_los)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(p).batch(128)

In [ ]:
dataset

In [ ]:
earl=EarlyStopping(patience=33,monitor="g_loss",mode="min")
hist = bamgan.fit( dataset, epochs=1200, callbacks=[earl])

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['d_loss'], label='d_loss')
plt.plot(hist.history['g_loss'], label='g_loss')
plt.legend()
plt.show()

In [ ]:
l=generator.predict(tf.random.normal((128,128)))

In [ ]:
def decoder(encode):
    smiles=""
    encode=(encode>0.5).astype(int)
    en=encode.reshape(239,45)
    for aw,i in enumerate(en):
        p=-1
        nu=0
        for d,j in enumerate(i):
            if(j==1):
                p=d
                nu=nu+1
        if nu>1:
            break
        
        if(p!=-1):
            for l,k in AtomExist.items():
                if(k==p):
                    smiles=smiles+l
                    break
    for key,rem in remp.items():
        smiles=smiles.replace(rem,key)
    return smiles


In [ ]:
for i in range(128):
    print(decoder(l[i]))

In [ ]:
for i in range(40):
    print(data1.Smiles.iloc[i])

# Another: codage  + Model LSTM :pour un molecule Predict 

In [ ]:
d=data1['Smiles']

In [ ]:
smile=d.to_frame()

In [ ]:
print(type(smile))
smile.shape      

In [ ]:
#Building Model RNN using SMILES
chars=list(d)
type(chars)

# SMILES MAPPING

In [ ]:

from collections import OrderedDict
from itertools import chain

from sklearn.utils import shuffle


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM

In [ ]:
df_sampled =d.sample(n=10000)

In [ ]:
df3=data.copy().head(5000)
k=df3['Smiles'].astype('<U53')
k

In [ ]:
train=df_sampled.values
trainf=train.astype('<U53')

In [ ]:
trainf

In [ ]:
Chem.MolFromSmiles(trainf[3])

In [ ]:
# creating mapping for each char to integer, also mapping for the E (end) is manually inserted into the dictionaries.
unique_chars = sorted(list(OrderedDict.fromkeys(chain.from_iterable(trainf))))
# maps each unique character as int
char_to_int = dict((c, i) for i, c in enumerate(unique_chars))

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(unique_chars))

In [ ]:
char_to_int

In [ ]:
int_to_char

In [ ]:
# add stop letter to dictionary
char_to_int.update({"E" : len(char_to_int)})
int_to_char.update({len(int_to_char) : "E"})

In [ ]:
# how many unique characters do we have?
mapping_size = len(char_to_int)
reverse_mapping_size = len(int_to_char)
print ("Size of the character to integer dictionary is: ", mapping_size)
print ("Size of the integer to character dictionary is: ", reverse_mapping_size)

In [ ]:
# Generate the datasets
def gen_data(data, int_to_char, char_to_int, embed):
    
    one_hot =  np.zeros((data.shape[0], embed+1, len(char_to_int)),dtype=np.int8)
    for i,smile in enumerate(data):
        #encode the chars
        for j,c in enumerate(smile):
            one_hot[i,j,char_to_int[c]] = 1
        #Encode endchar
        one_hot[i,len(smile):,char_to_int["E"]] = 1
    #Return two, one for input and the other for output
    return one_hot[:,0:-1,:], one_hot[:,1:,:]

In [ ]:
# get longest sequence
embed = max([len(seq) for seq in trainf])

# Get datasets
X, Y = gen_data(trainf, int_to_char, char_to_int, embed)
X, Y = shuffle(X, Y)


In [ ]:
type(X)

In [ ]:
type(Y)

In [ ]:
!pip install --upgrade tensorflow==2.7

# LSTM MODEL

In [ ]:
"""CREATING THE LSTM MODEL """

# Create the model (simple 2 layer LSTM)
model = Sequential()
#None accepts any length of features
#number of unique features of the input #returns outputs to the hidden layers
model.add(LSTM(256, input_shape=(None, X.shape[2]), return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(256, return_sequences = True))
model.add(Dropout(0.25))
model.add(Dense(Y.shape[-1], activation='softmax'))
print (model.summary())

In [ ]:
# Compile the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam')
#multi-classification problem
# Fit the model
history = model.fit(X, Y, epochs = 50, batch_size = 256)

In [ ]:
# Store to not having to train again...
model.save_weights("./twolayerlstm")

# Load to continue training or evaluate...
model = model.load_weights("./twolayerlstm")

In [ ]:
"""Predictions"""

# Calculate predictions
predictions = model.predict(X, verbose=0)

# Compare to correct result
train_res = np.argmax(Y,axis=2)-np.argmax(predictions,axis=2)

In [ ]:
# Count correct and incorrect predictions
no_false = np.count_nonzero(train_res)
no_true = len(Y)*embed-no_false
print("Average success rate on training set: %s %%" %str(np.round(100*no_true/(embed*len(Y)),2)))

In [ ]:
# Take a look at the model predictions on the training set next to the true result

for i in range(40):
    v = model.predict(X[i:i+1]) 
    idxs = np.argmax(v, axis=2)
    pred=  "".join([int_to_char[h] for h in idxs[0]])
    
    
    idxs2 = np.argmax(Y[i:i+1], axis=2)
    true =  "".join([int_to_char[k] for k in idxs2[0]])
    if true != pred:
        print (true, pred)

# Pretrained Model EXample

In [ ]:



import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from rdkit import Chem

tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")

def generate_smiles(template, num_samples=10):
    input = f"{template} <mask>"
    input_ids = tokenizer.encode(input, return_tensors='pt')
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    output = model(input_ids).logits
    mask_token_logits = output[0, mask_token_index, :]
    top_k_tokens = torch.topk(mask_token_logits, k=num_samples, dim=1).indices[0].tolist()

    smiles = []
    for token in top_k_tokens:
        token_str = tokenizer.decode([token])
        smile = input.replace(tokenizer.mask_token, token_str).replace(" ", "")
        if is_valid_smiles(smile):
            smiles.append(smile)
        if len(smiles) == num_samples:
            break

    return smiles

def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return False
    return True

# example usage
template = "Cc1ccc(cc1N)C(=O)NCCN"
smiles = generate_smiles(template)
print(smiles)

In [ ]:
import requests

# Define a SMILES string to search for
smiles = 'Cc1ccc(cc1N)C(=O)NCCNSN'

# Send a request to the PubChem API to search for the molecule
url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{smiles}/cids/TXT'
response = requests.get(url)

# Check if the response contains any CIDs (Compound IDs)
if response.status_code == 200 and response.text != '':
    cids = [int(cid) for cid in response.text.split()]
    print(f'The SMILES string "{smiles}" matches {len(cids)} known molecules in PubChem.')
else:
    print(f'The SMILES string "{smiles}" does not match any known molecules in PubChem.')


In [ ]:
smiles = 'Cc1ccc(cc1N)C(=O)NCCNSN'

# Send a request to the ChemSpider API to search for the molecule
url = f'https://api.rsc.org/compounds/v1/filter/smiles/{smiles}/ids'
headers = {'apikey': '<YOUR_CHEMSPIDER_API_KEY>'}
response = requests.get(url, headers=headers)

# Check if the response contains any IDs
if response.status_code == 200 and response.json()['count'] > 0:
    ids = response.json()['ids']
    print(f'The SMILES string "{smiles}" matches {len(ids)} known molecules in ChemSpider.')
else:
    print(f'The SMILES string "{smiles}" does not match any known molecules in ChemSpider.')
"Cc1ccc(cc1N)C(=O)NCCN"